In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('CKD.csv')

In [3]:
data

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [30]:
data.shape

(399, 28)

In [4]:
data[['rbc','pc','pcc','ba']].value_counts()

rbc       pc        pcc         ba        
normal    normal    notpresent  notpresent    294
          abnormal  notpresent  notpresent     20
                    present     notpresent     18
abnormal  normal    notpresent  notpresent     16
          abnormal  notpresent  notpresent     15
normal    abnormal  present     present         8
abnormal  abnormal  present     notpresent      7
                    notpresent  present         6
normal    normal    present     notpresent      6
                    notpresent  present         3
          abnormal  notpresent  present         2
abnormal  normal    notpresent  present         1
                    present     notpresent      1
                                present         1
normal    normal    present     present         1
dtype: int64

In [5]:
data=pd.get_dummies(data,drop_first=True)

In [6]:
data.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [7]:
independence=data[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependence=data[['classification_yes']]

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independence,dependence,test_size=1/3,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

## Decision Tree Classifier

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini', 'entropy', 'log_loss'],'splitter':['best', 'random'],'max_features':['sqrt', 'log2']}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [12]:
y_pred=grid.predict(X_test)


In [13]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [14]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [15]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [17]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [20]:
print(f'Decision Tree Classifier Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}')


Decision Tree Classifier Confusion Matrix:
[[50  1]
 [ 1 81]]
Classification_report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        51
           1       0.99      0.99      0.99        82

    accuracy                           0.98       133
   macro avg       0.98      0.98      0.98       133
weighted avg       0.98      0.98      0.98       133

F1 score for the best parameter is {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'random'} and 
F1 score value is 0.9849624060150376
AUC Score is 0.9840985174557627


## Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid={'penalty':['l2'],'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],'max_iter': [100, 200, 300, 1000,20000]}
grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


C:\Users\Arun\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Arun\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'max_iter': [100, 200, 300, 1000, 20000],
                         'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [11]:
y_pred=grid.predict(X_test)

In [12]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [13]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [14]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [15]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [16]:
from sklearn.model_selection import cross_val_score, KFold
cross_scores = cross_val_score(grid, X_train, y_train.values.ravel(), cv=5, scoring='f1')


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits


C:\Users\Arun\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [18]:
print(f'Logistic Regression Classifier Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}\n and Cross_val_score is{cross_scores}')


Logistic Regression Classifier Confusion Matrix:
[[51  0]
 [ 1 81]]
Classification_report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133

F1 score for the best parameter is {'max_iter': 100, 'penalty': 'l2', 'solver': 'sag'} and 
F1 score value is 0.9924946382275899
AUC Score is 1.0
 and Cross_val_score is[0.98507463 0.98507463 0.98461538 1.         1.        ]


## Support Vector Classification

In [19]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid={'C':[1,50,100,10,200],'kernel':['linear', 'poly', 'rbf', 'sigmoid']}
grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Users\Arun\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [1, 50, 100, 10, 200],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [20]:
y_pred=grid.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [22]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [23]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [24]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [26]:
from sklearn.model_selection import cross_val_score, KFold
cross_scores = cross_val_score(grid, X_train, y_train.values.ravel(), cv=5, scoring='f1')


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [28]:
print(f'Support Vector Classifier Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}\n and Cross_val_score is{cross_scores}')


Support Vector Classifier Confusion Matrix:
[[51  0]
 [ 1 81]]
Classification_report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133

F1 score for the best parameter is {'C': 10, 'kernel': 'sigmoid'} and 
F1 score value is 0.9924946382275899
AUC Score is 1.0
 and Cross_val_score is[0.98507463 0.98550725 1.         0.98507463 0.98461538]


## Random Forest Classifer

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_estimators':[10,100,1000],'criterion':['gini', 'entropy', 'log_loss'],'max_features':['sqrt', 'log2'],'class_weight':['balanced', 'balanced_subsample']}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 100, 1000]},
             scoring='f1_weighted', verbose=3)

In [52]:
y_pred=grid.predict(X_test)

In [53]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [54]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [55]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [56]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [57]:
print(f'Random Forest Classifier Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}')


Random Forest Classifier Confusion Matrix:
[[51  0]
 [ 2 80]]
Classification_report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        51
           1       1.00      0.98      0.99        82

    accuracy                           0.98       133
   macro avg       0.98      0.99      0.98       133
weighted avg       0.99      0.98      0.99       133

F1 score for the best parameter is {'class_weight': 'balanced_subsample', 'criterion': 'log_loss', 'max_features': 'log2', 'n_estimators': 10} and 
F1 score value is 0.9850141736106648
AUC Score is 0.999402199904352


## K nearest neightbour

In [58]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[1,3,5,7,10],'weights':['uniform', 'distance'],'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],'p':[1,2]}
grid=GridSearchCV(KNeighborsClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 3, 5, 7, 10], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [59]:
y_pred=grid.predict(X_test)

In [60]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [61]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [62]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [63]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [64]:
print(f'KNN Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}')


KNN Confusion Matrix:
[[51  0]
 [ 5 77]]
Classification_report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        51
           1       1.00      0.94      0.97        82

    accuracy                           0.96       133
   macro avg       0.96      0.97      0.96       133
weighted avg       0.97      0.96      0.96       133

F1 score for the best parameter is {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'} and 
F1 score value is 0.9626932787797391
AUC Score is 0.9695121951219512


## Naive Bayes

In [67]:
###GaussianNB

from sklearn.naive_bayes  import GaussianNB
from sklearn.model_selection import GridSearchCV
param_grid = {}

grid=GridSearchCV(GaussianNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(estimator=GaussianNB(), n_jobs=-1, param_grid={},
             scoring='f1_weighted', verbose=3)

In [68]:
y_pred=grid.predict(X_test)

In [69]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [70]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [71]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [72]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [73]:
print(f'Gaussian NB Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}')


Gaussian NB Confusion Matrix:
[[51  0]
 [ 3 79]]
Classification_report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        51
           1       1.00      0.96      0.98        82

    accuracy                           0.98       133
   macro avg       0.97      0.98      0.98       133
weighted avg       0.98      0.98      0.98       133

F1 score for the best parameter is {} and 
F1 score value is 0.9775556904684072
AUC Score is 1.0


In [82]:
###Multinomial NB
independence=data[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependence=data[['classification_yes']]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independence,dependence,test_size=1/3,random_state=0)

from sklearn.naive_bayes  import MultinomialNB
from sklearn.model_selection import GridSearchCV
param_grid = {}  # Whether to learn class prior probabilities or not
y_train=y_train.values.ravel()
grid=GridSearchCV(MultinomialNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(estimator=MultinomialNB(), n_jobs=-1, param_grid={},
             scoring='f1_weighted', verbose=3)

In [83]:
y_pred=grid.predict(X_test)

In [84]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [85]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [86]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [87]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [88]:
print(f'Multinomial NB Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}')


Multinomial NB Confusion Matrix:
[[50  1]
 [23 59]]
Classification_report:
              precision    recall  f1-score   support

           0       0.68      0.98      0.81        51
           1       0.98      0.72      0.83        82

    accuracy                           0.82       133
   macro avg       0.83      0.85      0.82       133
weighted avg       0.87      0.82      0.82       133

F1 score for the best parameter is {} and 
F1 score value is 0.8215780250262184
AUC Score is 0.9151123864179818


In [93]:
##ComplementNB
independence=data[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependence=data[['classification_yes']]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independence,dependence,test_size=1/3,random_state=0)
from sklearn.naive_bayes  import ComplementNB
from sklearn.model_selection import GridSearchCV
param_grid = {}  # Whether to learn class prior probabilities or not
y_train=y_train.values.ravel()
grid=GridSearchCV(ComplementNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(estimator=ComplementNB(), n_jobs=-1, param_grid={},
             scoring='f1_weighted', verbose=3)

In [94]:
y_pred=grid.predict(X_test)

In [95]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [96]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [97]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [98]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [99]:
print(f'Complement NB Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}')


Complement NB Confusion Matrix:
[[50  1]
 [23 59]]
Classification_report:
              precision    recall  f1-score   support

           0       0.68      0.98      0.81        51
           1       0.98      0.72      0.83        82

    accuracy                           0.82       133
   macro avg       0.83      0.85      0.82       133
weighted avg       0.87      0.82      0.82       133

F1 score for the best parameter is {} and 
F1 score value is 0.8215780250262184
AUC Score is 0.9151123864179818


In [100]:
#BernoulliNB

independence=data[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependence=data[['classification_yes']]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independence,dependence,test_size=1/3,random_state=0)
from sklearn.naive_bayes  import BernoulliNB
from sklearn.model_selection import GridSearchCV
param_grid = {}  # Whether to learn class prior probabilities or not
y_train=y_train.values.ravel()
grid=GridSearchCV(BernoulliNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1, param_grid={},
             scoring='f1_weighted', verbose=3)

In [101]:
y_pred=grid.predict(X_test)

In [102]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [103]:
from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)

In [104]:
from sklearn.metrics import f1_score
f1_macros=f1_score(y_test,y_pred,average='weighted')

In [105]:
from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:, 1])

In [106]:
print(f'BernoulliNB  Confusion Matrix:\n{cm}\nClassification_report:\n{cf_model}\nF1 score for the best parameter is {grid.best_params_} and \nF1 score value is {f1_macros}\nAUC Score is {auc_score}')


BernoulliNB  Confusion Matrix:
[[51  0]
 [ 8 74]]
Classification_report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.93        51
           1       1.00      0.90      0.95        82

    accuracy                           0.94       133
   macro avg       0.93      0.95      0.94       133
weighted avg       0.95      0.94      0.94       133

F1 score for the best parameter is {} and 
F1 score value is 0.9404945931261721
AUC Score is 0.9966523194643712
